## Wind directions from Kepert BL model

### Fit on real SAR cases

### Step-by-step convergence model to cope the fact that the reduction factor is on the tangential wind speed and not the total one (doesn't change anything)

### Optimization to find the minimum of asymmetric differences via Scipy (takes too long)

TODO

- In Kepert framework, the TC moves to the right. Put everything moving to the top of the page to be consistent. WAIT: I feel like in Theo's TCVA database TCs are also moving to the right of the image.

- In Kepert framework, we have the storm-relative wind speed. To convert it to a wind speed that would be observed by a radar, add the storm motion. In scatterometer acquisitions, the strongest surface winds appear on the right side (see Tamizi). So consistent with what I found.

- The BL has a time 1/I to set up. So near the Rmax this is roughly 2-min, very close to the 1-min wind speed of SAR data. Yet near R+ this is more like 2-hrs, so I would need to average (~40km) to get wind speed close to that of SAR. Yet, this doesn't change too much wind_speed as a function of theta, because near R+ the wind speed gradients are very small. Yet it justifies that asymmetries from lower-resolution images (such as Rad or Ascat) near R+ may be examined to do similar work.

- Make the model more user-friendly!!!!

- Try on TC Goni? (to link with Paper 3)

- Link with h+ and u_bar from Paper 3?

- If I do a composite analysis, remove storms from the southern hemisphere

- To re-put the kepert asymmetries at the right location, turn the Kepert wind field from an angle phi, where phi is the phase of the fitted cosine.

- I need to check that the cosine fit happened well at each radius. Remove all the cases for which this was not the case. Indeed, because of rain or spirals, I will have holes in the azimuthal profile of wind_speed at certain radii, which makes the cosine not fit well. I can probably compute a metric such as the RMSE between the fitted cosine and the real data, and remove all the cases that have a too large RMSE (even if that's a lot of cases). In the end if I have only 20 cases left, this is good, because I will make a law on K with the SST. Not that considering the difference between the 90% and 10% percentiles is not necessarily more robust (it is more robust in some cases, but not a significant nb of cases, and I would like to keep track of the phase of the fitted cosine to put the wind field in the right direction).

- CAVEAT, the radius range where Kepert's model is valid is in fact very limited and confined near Rmax and R+;

WHYs

- To compute the WN1 in Kepert at R+, I need to use the R+ of SAR. This is because sharp gradients in the vorticity profiles when I ~ V/r, which prevents from efficiently computing R+. So I will probably also have to use the Rmax of SAR to be consistent.

- If I remove the symmetric part before computing the asymmetries, it doesn't change anything.

- Compared to the Holland wind profile, Willoughby doesn't seem to remove the I~V/r sharp gradients.

- Problem: the wind reduction factor is only on the total wind, not the tangential wind. So I will have to initialize using the wind reduction factor on the total wind, find the inflow angle, and then use these inflow angle values to compute the tangential wind, on which I reapply the wind reduction factor, and so on until the model converges. Answer: This is almost the same like computing the wind reduction factor on the total wind in the radii range for which the model is valid (inertial stability)

In [1]:
# General
import glob
import re
import os.path
# import warnings
# warnings.filterwarnings('ignore')
from tqdm.notebook import tqdm
import copy

# Arrays & Displays
import xarray as xr
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.gridspec import GridSpec
import matplotlib.cm as cm
import pandas as pd

# Data treatment
from datetime import datetime
from scipy.optimize import curve_fit
from scipy.optimize import minimize
from scipy.interpolate import griddata

# Statistics
from sklearn import linear_model, neighbors
from pykalman import KalmanFilter

# Default parameters
mpl.rcParams.update({'font.size': 18})
mpl.rcParams['figure.figsize'] = (15, 10)
mpl.rcParams['axes.facecolor'] = 'white'
mpl.rcParams['figure.facecolor'] = 'white'

In [2]:
#########################
### General functions ###
#########################
def coriolis(lat):
    '''Latitude must be in degrees.'''
    Omega = 7.2921e-5                             # Earth rotation vector
    fcor  = 2 * Omega * np.sin(lat * np.pi / 180) # Coriolis parameter assuming it's constant 
    return fcor

# def h80(r, vm, rm, B, lat, vmin=0):
#     '''Holland 1980 profile.'''
#     fcor = coriolis(lat)
#     V    = r * 0.
#     V    = vmin + np.sqrt((vm ** 2) * ((rm / r) ** B) * np.exp(1 - (rm / r) ** B) + (r * fcor / 2) ** 2) - (r * fcor / 2)
#     return V

def holland_profile(r, lat, B, Vmin, Rmax, Vmax):
    '''We assume that rho is constant and equals 1.15 kg.m-3'''
    fcor   = abs(coriolis(lat))
    rho    = 1.15
    r      = r.astype(float) + 0.001           # To avoid dividing by zero
    r_star = (Rmax / r) ** B
    V      = r * 0.
    V      = Vmin + np.sqrt( ((Vmax - Vmin) ** 2) * r_star * np.exp(1 - r_star) + (r * fcor / 2) ** 2) - (r * fcor / 2)
    return V

def pol2cart(rho, phi):
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    return(x, y)

#########################
### KEPERT 2001 MODEL ###
#########################
# Inertial stability
def I(Rs, Vs, fcor):
    dr   = np.diff(Rs[0, :])[0] # meters
    ksi  = 2 * Vs / Rs + fcor
    zeta = np.gradient(Vs, dr, axis=1) + Vs / Rs + fcor
    return np.sqrt(ksi * zeta)

# Coefficients
def sqrt_alpha_beta(Rs, Vs, fcor):
    '''Corresponds to sqrt(alpha / beta).'''
    dr   = np.diff(Rs[0, :])[0] # meteRs
    ksi  = 2 * Vs / Rs + fcor
    zeta = np.gradient(Vs, dr, axis=1) + Vs / Rs + fcor
    return np.sqrt(ksi / zeta)

def eta(Rs, Vs, K, C, fcor):
    return C * Vs * np.sqrt(2 / (K * ((Vs / Rs) + I(Rs, Vs, fcor))))

def ki(Rs, Vs, K, C, fcor):
    return C * Vs * np.sqrt(2 / (K * I(Rs, Vs, fcor)))

def psi(Rs, Vs, K, C, fcor):
    return C * Vs * np.sqrt(2 / abs(K * ((Vs / Rs) - I(Rs, Vs, fcor))))

# Amplitudes
def A0(Rs, Vs, K, C, fcor):
    num = -ki(Rs, Vs, K, C, fcor) * Vs * [1 + 1j * (1 + ki(Rs, Vs, K, C, fcor))]
    den = 2 * ki(Rs, Vs, K, C, fcor) ** 2 + 3 * ki(Rs, Vs, K, C, fcor) + 2
    return num / den

def A1(Rs, Vs, Ut, K, C, fcor):
    num = -eta(Rs, Vs, K, C, fcor) * Ut * [1 - 2 * sqrt_alpha_beta(Rs, Vs, fcor) + (1 + 1j) * (1 - sqrt_alpha_beta(Rs, Vs, fcor)) * psi(Rs, Vs, K, C, fcor)]
    den = sqrt_alpha_beta(Rs, Vs, fcor) * [(2 + 2j) * (1 + eta(Rs, Vs, K, C, fcor) * psi(Rs, Vs, K, C, fcor)) + 3 * eta(Rs, Vs, K, C, fcor) + 3j * psi(Rs, Vs, K, C, fcor)]
    return num / den

def A_1(Rs, Vs, Ut, K, C, fcor):
    num = -psi(Rs, Vs, K, C, fcor) * Ut * [1 + 2 * sqrt_alpha_beta(Rs, Vs, fcor) + (1 + 1j) * (1 + sqrt_alpha_beta(Rs, Vs, fcor)) * eta(Rs, Vs, K, C, fcor)]
    den = sqrt_alpha_beta(Rs, Vs, fcor) * [(2 + 2j) * (1 + eta(Rs, Vs, K, C, fcor) * psi(Rs, Vs, K, C, fcor)) + 3 * psi(Rs, Vs, K, C, fcor) + 3j * eta(Rs, Vs, K, C, fcor)]
    return num / den

def A1_prime(Rs, Vs, Ut, K, C, fcor):
    num = -eta(Rs, Vs, K, C, fcor) * Ut * [1 - 2 * sqrt_alpha_beta(Rs, Vs, fcor) + (1 - 1j) * (1 - sqrt_alpha_beta(Rs, Vs, fcor)) * psi(Rs, Vs, K, C, fcor)]
    den = sqrt_alpha_beta(Rs, Vs, fcor) * [2 + 2j + 3 * (eta(Rs, Vs, K, C, fcor) + psi(Rs, Vs, K, C, fcor)) + (2 - 2j) * eta(Rs, Vs, K, C, fcor) * psi(Rs, Vs, K, C, fcor)]
    return num / den

def A_1_prime(Rs, Vs, Ut, K, C, fcor):
    num = -psi(Rs, Vs, K, C, fcor) * Ut * [1 + 2 * sqrt_alpha_beta(Rs, Vs, fcor) + (1 + 1j) * (1 + sqrt_alpha_beta(Rs, Vs, fcor)) * eta(Rs, Vs, K, C, fcor)]
    den = sqrt_alpha_beta(Rs, Vs, fcor) * [2 - 2j + 3 * (eta(Rs, Vs, K, C, fcor) + psi(Rs, Vs, K, C, fcor)) + (2 + 2j) * eta(Rs, Vs, K, C, fcor) * psi(Rs, Vs, K, C, fcor)]
    return num / den

# Radial and tangential wind components at the surface (z = 0)
def u0(Rs, Vs, K, C, fcor):
    '''At the surface (z = 0)'''
    u0 = sqrt_alpha_beta(Rs, Vs, fcor) * np.real(A0(Rs, Vs, K, C, fcor))
    return np.reshape(u0,  np.shape(u0)[1:])

def v0(Rs, Vs, K, C, fcor):
    '''At the surface (z = 0)'''
    v0 = np.imag(A0(Rs, Vs, K, C, fcor))
    return np.reshape(v0,  np.shape(v0)[1:])

def u1(THs, Rs, Vs, Ut, K, C, fcor):
    '''At the surface (z = 0)'''
    u1        = Vs * 0
    msk       = I(Rs, Vs, fcor) >= Vs / Rs
    if_msked  = sqrt_alpha_beta(Rs, Vs, fcor) * np.real(A1(Rs, Vs, Ut, K, C, fcor)       * np.exp(1j * THs))
    otherwise = sqrt_alpha_beta(Rs, Vs, fcor) * np.real(A1_prime(Rs, Vs, Ut, K, C, fcor) * np.exp(1j * THs))
    u1[msk]   = np.reshape(if_msked,  np.shape(if_msked)[1:])[msk]
    u1[~msk]  = np.reshape(otherwise, np.shape(if_msked)[1:])[~msk]
    return u1

def v1(THs, Rs, Vs, Ut, K, C, fcor):
    '''At the surface (z = 0)'''
    v1        = Vs * 0
    msk       = I(Rs, Vs, fcor) >= Vs / Rs
    if_msked  = np.imag(A1(Rs, Vs, Ut, K, C, fcor)       * np.exp(1j * THs))
    otherwise = np.imag(A1_prime(Rs, Vs, Ut, K, C, fcor) * np.exp(1j * THs))
    v1[msk]   = np.reshape(if_msked,  np.shape(if_msked)[1:])[msk]
    v1[~msk]  = np.reshape(otherwise, np.shape(if_msked)[1:])[~msk]
    return v1

def u_1(THs, Rs, Vs, Ut, K, C, fcor):
    '''At the surface (z = 0)'''
    u1        = Vs * 0
    msk       = I(Rs, Vs, fcor) >= Vs / Rs
    if_msked  = sqrt_alpha_beta(Rs, Vs, fcor) * np.real(A_1(Rs, Vs, Ut, K, C, fcor)       * np.exp(-1j * THs))
    otherwise = sqrt_alpha_beta(Rs, Vs, fcor) * np.real(A_1_prime(Rs, Vs, Ut, K, C, fcor) * np.exp(-1j * THs))
    u1[msk]   = np.reshape(if_msked,  np.shape(if_msked)[1:])[msk]
    u1[~msk]  = np.reshape(otherwise, np.shape(if_msked)[1:])[~msk]
    return u1

def v_1(THs, Rs, Vs, Ut, K, C, fcor):
    '''At the surface (z = 0)'''
    v1        = Vs * 0
    msk       = I(Rs, Vs, fcor) >= Vs / Rs
    if_msked  = np.imag(A_1(Rs, Vs, Ut, K, C, fcor)       * np.exp(-1j * THs))
    otherwise = np.imag(A_1_prime(Rs, Vs, Ut, K, C, fcor) * np.exp(-1j * THs))
    v1[msk]   = np.reshape(if_msked,  np.shape(if_msked)[1:])[msk]
    v1[~msk]  = np.reshape(otherwise, np.shape(if_msked)[1:])[~msk]
    return v1

# TOTAL WIND COMPONENTS IN THE BL
def u_BL(THs, Rs, Vs, Ut, K, C, fcor):
    '''At the surface (z = 0)'''
    return u_1(THs, Rs, Vs, Ut, K, C, fcor) + u0(Rs, Vs, K, C, fcor) + u1(THs, Rs, Vs, Ut, K, C, fcor)

def v_BL(THs, Rs, Vs, Ut, K, C, fcor):
    '''At the surface (z = 0)'''
    return Vs + v_1(THs, Rs, Vs, Ut, K, C, fcor) + v0(Rs, Vs, K, C, fcor) + v1(THs, Rs, Vs, Ut, K, C, fcor)

# FINAL MODEL
def kepert2001(THs, Rs, Vs, Ut, K, C, fcor):
    # Returns Kepert wind speeds
    u_K01   = u_BL(THs, Rs, Vs, Ut, K, C, fcor)
    v_K01   = v_BL(THs, Rs, Vs, Ut, K, C, fcor)  

    # Adds the translation speed
    Utu     = Ut * np.cos(THs) # Radial component of Ut
    Utv     = -Ut * np.sin(THs) # Tangential component of Ut
    
    return u_K01 + Utu, v_K01 + Utv

#########################
### CREATE STRUCTURES ###
#########################
def get_xrDataset_polar(rs, ths, u_K01, v_K01):
    kep_pol = xr.Dataset(
        data_vars={
            'wind_speed': (('theta', 'rad'), np.sqrt(v_K01 ** 2 + u_K01 ** 2)),
            'rad_wind':   (('theta', 'rad'), u_K01),
            'tan_wind':   (('theta', 'rad'), v_K01),
        },
        coords={
            'theta': np.rad2deg(ths),
            'rad': rs,
        }
    )
    kep_pol['inflow_angle'] = np.rad2deg(np.arctan((kep_pol.rad_wind / kep_pol.tan_wind))) # Inflow Angle Polar
    return kep_pol

def get_xrDataset_cartesian(Xs, Ys, x_ref, y_ref, dim_ref, kep_pol):
    '''Slower than its polar counterpart, because 4 interpolations are performed.'''
    kep_car = xr.Dataset(
        data_vars={
            'wind_speed':   (('x', 'y'), griddata((Xs.flatten(), Ys.flatten()), np.array(kep_pol.wind_speed).flatten(),   (x_ref, y_ref), method='nearest')),
            'rad_wind':     (('x', 'y'), griddata((Xs.flatten(), Ys.flatten()), np.array(kep_pol.rad_wind).flatten(),     (x_ref, y_ref), method='nearest')),
            'tan_wind':     (('x', 'y'), griddata((Xs.flatten(), Ys.flatten()), np.array(kep_pol.tan_wind).flatten(),     (x_ref, y_ref), method='nearest')),
            'inflow_angle': (('x', 'y'), griddata((Xs.flatten(), Ys.flatten()), np.array(kep_pol.inflow_angle).flatten(), (x_ref, y_ref), method='nearest')),
        },
        coords={
            'x': dim_ref,
            'y': dim_ref,
        }
    )
    return kep_car

#########################
###   MODEL FITTING   ###
#########################
def wind_reduction_factor(Rs, Vs, K, C, fcor):
    '''wrf = surface azim. wind / gradient azim. wind'''
    ki_cst  = ki(Rs, Vs, K, C, fcor)
    num     = ki_cst ** 2 + 2 * ki_cst + 2
    den     = 2 * ki_cst ** 2 + 3 * ki_cst + 2
    return num / den

def wn1(thetas, a, phi, c):
    return a * np.cos(thetas + phi) + c

def get_wn1(thetas, wind_speed, initial_condition=15, upper_bound=80):
    '''
    initial_condition: initial conidition for the amplitude and the constant c (just put Vplus)
    upper_bound: upper bound for the the amplitude and the constant c (just put Vmax)
    '''
    # Fitting process
    popt, pcov = curve_fit(
        f=wn1,
        xdata=thetas,
        ydata=wind_speed,
        p0=[initial_condition, np.pi / 2, initial_condition],
        bounds=([1., 0., 0.1], [upper_bound, np.pi, upper_bound]) # ([lower bounds], [upper bounds])
    )
    a, phi, c = popt[0], popt[1], popt[2]
    return a, phi, c

def get_kepert_asymmetries(THs, Rs, ths, rs, Vs, Ut, K, C, fcr):
    # Estimate the wind at the top of the BL with the wind reduction factor
    wrf    = wind_reduction_factor(Rs, Vs, K, C, fcr) # azimuthal wind
    # wrf    = wind_reduction_factor_total_wind(Rs, Vs, K, C, fcr, Ut) # total wind
    Vs_try = Vs / wrf
    
    # Apply Kepert's model
    u_K01, v_K01 = kepert2001(THs, Rs, Vs, Ut, K, C, fcr) # Takes 3-4 seconds
    kep_pol      = get_xrDataset_polar(rs, ths, u_K01, v_K01)
    
    # Test
    ASS = []
    for rad in np.linspace(ds.rmx - 2000, 2 * ds.rps, 20): # WTF? TO UPDATE: ds should be as input parameters of the function (and change its name to avoid errors)
        as_rmx, _,  _ = get_wn1(np.deg2rad(kep_pol.theta), kep_pol.wind_speed.sel(rad=rad, method='nearest'), float(ds.vps), float(ds.vmx))
        ASS.append(as_rmx)
    plt.plot(np.linspace(ds.rmx - 2000, 2 * ds.rps, 20), ASS, label='K = {}'.format(K))
    
    # kep_pol.wind_speed.sel(rad=float(ds.rmx), method='nearest').plot(label='K = {}'.format(K))
    
    # # Plot asymmetries
    # kep_pol.wind_speed.sel(rad=float(ds.rmx), method='nearest').plot(label='K = {}'.format(K))
    # kep_pol.wind_speed.sel(rad=float(ds.rps), method='nearest').plot(label='K = {}'.format(K))
    
    # Extract WN1 metrics
    ak_rmx, phi, ck_rmx = get_wn1(np.deg2rad(kep_pol.theta), kep_pol.wind_speed.sel(rad=float(ds.rmx), method='nearest'), float(ds.vps), 2 * float(ds.vmx)) # Kepert at Rmax
    # print(ak_rmx, phi, ck_rmx)
    # plt.plot(kep_pol.theta, wn1(np.deg2rad(kep_pol.theta), ak_rmx, phi, ck_rmx))
    ak_rps, _, ck_rps = get_wn1(np.deg2rad(kep_pol.theta), kep_pol.wind_speed.sel(rad=float(ds.rps), method='nearest'), float(ds.vps), float(ds.vmx)) # Kepert at R+
    
    return ak_rmx, ck_rmx, ak_rps, ck_rps

In [3]:
#########################
###  HYPERPARAMETERS  ###
#########################
# Reference grids
rs           = np.linspace(100, 5e5, 5000)
ths          = np.linspace(0, 2 * np.pi, 361)                         # in radians
Rs, THs      = np.meshgrid(rs, ths)                                   # shape (361, 5000)
Xs, Ys       = pol2cart(Rs, THs)                                      # to create cartesian dataset
dim_ref      = np.linspace(-500, 500, int((500 // 1 * 2) + 1)) * 1000 # to create cartesian dataset
x_ref, y_ref = np.meshgrid(dim_ref, dim_ref)                          # to create cartesian dataset

In [4]:
### OPEN THE SAR IMAGE
# Need to choose a TC for which speed is sufficiently high during most intense phases of the TC
# If possible >2023 to have RCM
# I would like to have a collocated Ascat on the RCM acquisition!!

dsg      = xr.open_dataset('/home/arthur/results/sar_extended/03_addIBT/dsg.nc')
vls, cts = np.unique(dsg.name, return_counts=True)
print(vls[cts >= 10])

dsg.where(dsg.name == 'DORA', drop=True).isel(time=7).file
ds   = dsg.where(dsg.name == 'DORA', drop=True).isel(time=7)
# a   = xr.open_dataset('/home/arthur/data/cyclobs/RCM/sar_files/s1a-ew-owi-ca-20230809t165107-20230809t165210-000003-05FD3D_sw_ep052023_cyclone_polar.nc').isel(time=0) # time = 8, Dora
dsp  = xr.open_dataset('/home/arthur/data/cyclobs/RCM/sar_files/rcm1--owi-ca-20230809t050027-20230809t050143-00003-______sw_ep052023_cyclone_polar.nc').isel(time=0) # time = 7, Dora

/home/arthur/anaconda3/envs/paramProfiles/lib/python3.8/site-packages/xarray/backends/plugins.py:71: RuntimeWarning: Engine 'cfgrib' loading failed:
/home/arthur/anaconda3/envs/paramProfiles/lib/python3.8/lib-dynload/../../libffi.so.8: version `LIBFFI_CLOSURE_7.0' not found (required by /home/arthur/anaconda3/envs/paramProfiles/lib/python3.8/site-packages/_cffi_backend.cpython-38-x86_64-linux-gnu.so)
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


['DARIAN' 'DORA' 'DORIAN' 'FREDDY' 'LEE' 'MARGOT' 'SURIGAE']


In [5]:
# ### Extract WN1 metrics on the SAR
# as_rmx, _, _, cs_rmx = get_wn1(np.deg2rad(dsp.theta), dsp.sel(rad=ds.rmx, method='nearest').wind_speed, float(ds.vps), float(ds.vmx)) # SAR at Rmax
# as_rps, _, _, cs_rps = get_wn1(np.deg2rad(dsp.theta), dsp.sel(rad=ds.rps, method='nearest').wind_speed, float(ds.vps), float(ds.vmx)) # SAR at R+

# ### Extract WN1 metrics on the SAR
# as_rmx, _, cs_rmx = get_wn1(np.deg2rad(dsp.theta), dsp.sel(rad=ds.rmx, method='nearest').wind_speed, float(ds.vps), float(ds.vmx)) # SAR at Rmax
# as_rps, _, cs_rps = get_wn1(np.deg2rad(dsp.theta), dsp.sel(rad=ds.rps, method='nearest').wind_speed, float(ds.vps), float(ds.vmx)) # SAR at R+

In [6]:
### Initialize
# Fixed parameters
vmx = float(ds.vmx_hol)
rmx = float(ds.rmx_hol)
vmn = float(ds.vmn_hol)
B   = float(ds.B_hol)
lat = float(ds.lat)
fcr = float(ds.fcor)
Ut  = float(ds.speed)

# Holland with fixed parameters
# V   = h80(rs, vmx, rmx, B, lat, vmn)
V   = holland_profile(rs, lat, B, vmn, rmx, vmx)
Vs  = np.stack([V for i in range(np.shape(THs)[0])]) # Shape (361, 5000)

# Error threshold when fitting WN1s
eps = 0.1

In [7]:
def get_wn1(thetas, wind_speed, initial_condition=15, upper_bound=80):
    '''
    initial_condition: initial conidition for the amplitude and the constant c (just put Vplus)
    upper_bound: upper bound for the the amplitude and the constant c (just put Vmax)
    '''
    # Fitting process
    popt, pcov = curve_fit(
        f=wn1,
        xdata=thetas,
        ydata=wind_speed,
        p0=[initial_condition, np.pi / 2, initial_condition],
        bounds=([0.1, 0., 0.1], [upper_bound, np.pi, upper_bound]) # ([lower bounds], [upper bounds])
    )
    a, phi, c = popt[0], popt[1], popt[2]
    return a, phi, c

def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [8]:
# Compute SAR asymmetries
I_Vr         = (I(Rs, Vs, fcr) - Vs / rs)[0, :]                # I - V/r
r_stt, r_end = rs[I_Vr <= 0][0], rs[I_Vr <= 0][-1] # Define the bounds
# Initialize
ass         = []
valid_radii = np.array(dsp.rad.where((dsp.rad >= r_stt) & (dsp.rad <= r_end), drop=True))
# Iterate over all radii
for rad in tqdm(valid_radii): # Iterate only in the valid range to optimize time
    valid_dsp = dsp.sel(rad=rad, method='nearest').dropna(subset=['wind_speed'], dim='theta')
    a, phi, c = get_wn1(np.deg2rad(valid_dsp.theta), valid_dsp.wind_speed, float(ds.vps), float(ds.vmx))
    avg_ws    = float(valid_dsp.wind_speed.mean(dim='theta'))
    error     = float(rmse(wn1(np.deg2rad(valid_dsp.theta), a, phi, c), valid_dsp.wind_speed)) 
    if error < eps * avg_ws: # check that the fit did well
        ass.append(a)
    else:
        ass.append(np.nan)

  0%|          | 0/38 [00:00<?, ?it/s]

In [9]:
def get_kepert_asymmetries(THs, Rs, ths, rs, Vs, Ut, K, C, fcr, valid_radii):
    # Estimate the wind at the top of the BL with the wind reduction factor
    wrf          = wind_reduction_factor(Rs, Vs, K, C, fcr) # azimuthal wind
    Vs_try       = Vs / wrf

    # Apply Kepert's model
    u_K01, v_K01 = kepert2001(THs, Rs, Vs_try, Ut, K, C, fcr) # Takes 3-4 seconds
    kep_pol      = get_xrDataset_polar(rs, ths, u_K01, v_K01)

    # Compute asymmetries on Kepert wind field
    aks         = []
    # Iterate over all radii
    for rad in tqdm(valid_radii): # Iterate only in the valid range to optimize time
        ak, _, _  = get_wn1(np.deg2rad(kep_pol.theta), kep_pol.wind_speed.sel(rad=rad, method='nearest'), np.max(Vs[0, :]) / 3, np.max(Vs[0, :]))
        aks.append(ak)
    
    return aks, kep_pol

def get_asymmetries_difference(K, THs, Rs, ths, rs, Vs, Ut, C, fcr, valid_radii, ass):
    aks, _ = get_kepert_asymmetries(THs, Rs, ths, rs, Vs, Ut, K, C, fcr, valid_radii)
    diffs  = abs(np.array(aks) - np.array(ass))
    diff   = diffs.mean()
    print(diff)
    return diff

In [11]:
C = 1.5e-3

# Find K by minimizing the asymmetries difference;
# Takes a few minutes...
res = minimize(get_asymmetries_difference, x0=50, args=(THs, Rs, ths, rs, Vs, Ut, C, fcr, valid_radii, ass), tol=0.1, method='Nelder-Mead')

  0%|          | 0/38 [00:00<?, ?it/s]

2.040486349818045


  0%|          | 0/38 [00:00<?, ?it/s]

1.9675450677792539


  0%|          | 0/38 [00:00<?, ?it/s]

...

0.8110214481347368


  0%|          | 0/38 [00:00<?, ?it/s]

0.8110447191665602


  0%|          | 0/38 [00:00<?, ?it/s]

0.8110271915262557


In [12]:
res.x

array([231.71875])

In [ ]:
p1 = 's1a-ew-owi-cm-20201029t205631-20201029t205758-000003-0415BC_sw.nc'
p2 = 's1a-ew-owi-cm-20201030t092540-20201030t092711-000003-0415F8_sw.nc'
p3 = 'rs2--owi-cm-20201030t211604-20201030t211719-00003-______sw.nc'

In [ ]:
ds.file